In [ ]:
import torch
print(torch.cuda.get_device_name(0))

Tesla T4


In [ ]:
!pip install -U -q kaggle
!mkdir -p ~/.kaggle

In [ ]:
from google.colab import files
files.upload()

In [ ]:
                                            
                                            
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d manann/quotes-500k

 90% 81.0M/89.6M [00:01<00:00, 48.6MB/s]
100% 89.6M/89.6M [00:01<00:00, 62.0MB/s]


In [ ]:
!unzip quotes-500k.zip

Archive:  quotes-500k.zip
  inflating: quotes.csv              
  inflating: test.txt                
  inflating: train.txt               
  inflating: valid.txt               


## Prepare Quotes

Only quotes with tags 'inspirational','reality','optimism','life','motivational' are chosen for training the model


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
pd.set_option('display.max_colwidth', -1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


In [ ]:
data = pd.read_csv('quotes.csv')
d = data[:50000]


In [ ]:
#extract motivational quotes from the dataset

inspire = pd.DataFrame()
entry = pd.DataFrame()
tags = ['inspirational','reality','optimism','life','motivational']
for i in range (50000):
  for tag in tags:
    if(tag in d.iloc[[i],[2]].to_string()):
      if(len(d.iloc[[i],[2]].to_string().split(','))>2):
        taglist = d.iloc[[i],[2]].to_string().split(',')[1]
        entry['tag'] = [taglist]
        text = d['quote'][i]
        entry['quote'] = [text]
        inspire=inspire.append(entry)
        break
    

In [ ]:
print(inspire.shape)

(13871, 2)


In [ ]:
inspire.to_csv('bot.csv')

## Train Model

In [ ]:
!pip install transformers

In [ ]:
import csv
import os
import argparse
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import numpy as np
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import Dataset, DataLoader
from transformers import AdamW, get_cosine_with_hard_restarts_schedule_with_warmup, top_k_top_p_filtering
import warnings
import torch.nn.functional as F
import math, random
from tqdm import tqdm
warnings.filterwarnings('ignore')

In [ ]:
class MyDataset(Dataset):
  def __init__(self, data_file_name, tokenizer, seq_length=64):
    super().__init__()
    data_path = os.path.join(data_file_name)
    self.examples = []
    
    
    tag_tkn = tokenizer.additional_special_tokens_ids[0]
    quote_tkn = tokenizer.additional_special_tokens_ids[1]
    pad_tkn = tokenizer.pad_token_id
    eos_tkn = tokenizer.eos_token_id

    with open(data_path) as csv_file:
      csv_reader = csv.reader(csv_file, delimiter=',')
      
      for row in csv_reader:
        tag = [tag_tkn] + tokenizer.encode(row[1], max_length=seq_length//2-1)
        quote = [quote_tkn] + tokenizer.encode(row[2],max_length=seq_length//2-2) + [eos_tkn]
        tokens = tag + quote + [pad_tkn] * ( seq_length - len(tag) - len(quote) )
        segments = [tag_tkn] * len(tag) + [quote_tkn] * ( seq_length - len(tag) )
        labels = [-100] * (len(tag)+1) + quote[1:] + [-100] * ( seq_length - len(tag) - len(quote) )
        self.examples.append((tokens, segments, labels))

        
  def __len__(self):
    return len(self.examples)
      
  def __getitem__(self, item):
    
    return torch.tensor(self.examples[item])


In [ ]:
import numpy as np
from tqdm import tqdm


def fit(model, optimizer, train_dl, val_dl, epochs, device):

  for i in range(epochs):

    print('\n--- Starting epoch #{} ---'.format(i))

    model.train()

    # These 2 lists will keep track of the batch losses and batch sizes over one epoch:
    losses = []
    nums = []

    for xb in tqdm(train_dl, desc="Training"):
      # Move the batch to the training device:
      inputs = xb.to(device)

      # Call the model with the token ids, segment ids, and the ground truth (labels)
      outputs = model(inputs[:,0,:], token_type_ids=inputs[:,1,:], labels=inputs[:,2,:])
      
      # Add the loss and batch size to the list:
      loss = outputs[0]
      losses.append(loss.item())
      nums.append(len(xb))

      loss.backward()

      optimizer.step()
      model.zero_grad()

    # Compute the average cost over one epoch:
    train_cost = np.sum(np.multiply(losses, nums)) / sum(nums)


    # Now do the same thing for validation:

    model.eval()
    
    with torch.no_grad():
      losses = []
      nums = []

      for xb in tqdm(val_dl, desc="Validation"):
        inputs = xb.to(device)
        outputs = model(inputs[:,0,:], token_type_ids=inputs[:,1,:], labels=inputs[:,2,:])
        losses.append(outputs[0].item())
        nums.append(len(xb))

    val_cost = np.sum(np.multiply(losses, nums)) / sum(nums)

    print('\n--- Epoch #{} finished --- Training cost: {} / Validation cost: {}'.format(i, train_cost, val_cost))
  return model

In [ ]:
import math, random

from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

dataset = MyDataset('bot.csv',tokenizer)
# Create data indices for training and validation splits:

indices = list(range(len(dataset)))

random.seed(42)
random.shuffle(indices)

split = math.floor(0.1 * len(dataset))
train_indices, val_indices = indices[split:], indices[:split]

# Build the PyTorch data loaders:

train_sampler = SubsetRandomSampler(train_indices)
val_sampler = SubsetRandomSampler(val_indices)

train_loader = DataLoader(dataset, batch_size=32, sampler=train_sampler)
val_loader = DataLoader(dataset, batch_size=64, sampler=val_sampler)
# Note: we can double the batch size for validation since no backprogation is involved (thus it will fit on GPU's memory)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [ ]:

print ('Loading/Downloading GPT-2 Model')
TOKENIZER = GPT2Tokenizer.from_pretrained('distilgpt2')
MODEL = GPT2LMHeadModel.from_pretrained('distilgpt2')



In [ ]:
SPECIAL_TOKENS_DICT = {
    'pad_token': '<pad>',
    'additional_special_tokens': ['<tag>', '<quote>'],
}


BATCH_SIZE = 32
EPOCHS = 2
LEARNING_RATE = 2e-5
WARMUP_STEPS = 4
MAX_SEQ_LEN = 32
MODEL_NAME = 'bot'
DATA_FILE = 'bot.csv'
TOKENIZER.add_special_tokens(SPECIAL_TOKENS_DICT)
MODEL.resize_token_embeddings(len(TOKENIZER))
#TRAIN_LOADER, VAL_LOADER = get_data_loader(DATA_FILE, TOKENIZER)
DEVICE = 'cpu'
if torch.cuda.is_available():
	DEVICE = 'cuda'

Loading/Downloading GPT-2 Model


In [ ]:
from transformers import AdamW
MODEL.to(DEVICE)

# Fine-tune GPT2 for two epochs:
optimizer = AdamW(MODEL.parameters())
model = fit(MODEL, OPTIMIZER, train_loader, val_loader, epochs=2, device=DEVICE)

In [ ]:

import torch.nn.functional as F
from tqdm import trange

def top_k_top_p_filtering(logits, top_k=0, top_p=0.0, filter_value=-float('Inf')):
    """ Filter a distribution of logits using top-k and/or nucleus (top-p) filtering
        Args:
            logits: logits distribution shape (batch size x vocabulary size)
            top_k > 0: keep only top k tokens with highest probability (top-k filtering).
            top_p > 0.0: keep the top tokens with cumulative probability >= top_p (nucleus filtering).
                Nucleus filtering is described in Holtzman et al. (http://arxiv.org/abs/1904.09751)
        From: https://gist.github.com/thomwolf/1a5a29f6962089e871b94cbd09daf317
    """
    top_k = min(top_k, logits.size(-1))  # Safety check
    if top_k > 0:
        # Remove all tokens with a probability less than the last token of the top-k
        indices_to_remove = logits < torch.topk(logits, top_k)[0][..., -1, None]
        logits[indices_to_remove] = filter_value

    if top_p > 0.0:
        sorted_logits, sorted_indices = torch.sort(logits, descending=True)
        cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

        # Remove tokens with cumulative probability above the threshold
        sorted_indices_to_remove = cumulative_probs > top_p
        # Shift the indices to the right to keep also the first token above the threshold
        sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
        sorted_indices_to_remove[..., 0] = 0

        # scatter sorted tensors to original indexing
        indices_to_remove = sorted_indices_to_remove.scatter(dim=1, index=sorted_indices, src=sorted_indices_to_remove)
        logits[indices_to_remove] = filter_value
    return logits


# From HuggingFace, adapted to work with the context/slogan separation:
def sample_sequence(model, length, context, segments_tokens=None, num_samples=1, temperature=1, top_k=20, top_p=0.8, repetition_penalty=5,
                    device='cpu'):
    context = torch.tensor(context, dtype=torch.long, device=device)
    context = context.unsqueeze(0).repeat(num_samples, 1)
    generated = context

    with torch.no_grad():
        for _ in trange(length):

            inputs = {'input_ids': generated}
            if segments_tokens != None:
              inputs['token_type_ids'] = torch.tensor(segments_tokens[:generated.shape[1]]).unsqueeze(0).repeat(num_samples, 1)


            outputs = model(**inputs)  # Note: we could also use 'past' with GPT-2/Transfo-XL/XLNet/CTRL (cached hidden-states)
            next_token_logits = outputs[0][:, -1, :] / (temperature if temperature > 0 else 1.)

            # repetition penalty from CTRL (https://arxiv.org/abs/1909.05858)
            for i in range(num_samples):
                for _ in set(generated[i].tolist()):
                    next_token_logits[i, _] /= repetition_penalty
                
            filtered_logits = top_k_top_p_filtering(next_token_logits, top_k=top_k, top_p=top_p)
            if temperature == 0: # greedy sampling:
                next_token = torch.argmax(filtered_logits, dim=-1).unsqueeze(-1)
            else:
                next_token = torch.multinomial(F.softmax(filtered_logits, dim=-1), num_samples=1)
            generated = torch.cat((generated, next_token), dim=1)
    return generated


In [ ]:
tag = "love"

tag_tkn = tokenizer.additional_special_tokens_ids[0]
quote_tkn = tokenizer.additional_special_tokens_ids[1]

input_ids = [tag_tkn] + tokenizer.encode(tag)

segments = [quote_tkn] * 64

segments[:len(input_ids)] = [tag_tkn] * len(input_ids)

input_ids += [quote_tkn]

# Move the model back to the CPU for inference:
model.to(torch.device('cpu'))

# Generate 20 samples of max length 20
generated = sample_sequence(model, length=50, context=input_ids, segments_tokens=segments, num_samples=10)

print('\n\n--- Generated Slogans ---\n')
for g in generated:
  quote = tokenizer.decode(g.squeeze().tolist())
  quote = quote.split('<|endoftext|>')[0].split('<quote>')[1]
  print(quote)

100%|██████████| 50/50 [00:35<00:00,  1.40it/s]



--- Generated Slogans ---

 Love is like a magnet. The more you know, the greater love increases exponentially when it flows and becomes invisible as well with time again
 I love him, my friend. And I like to keep it and he always gets a lot of pleasure in me too but sometimes that is what really
 Love is like water. It flows and you can't see it coming in when there are no paths or even where the waves go until your friends leave
 We are the people we love, and when there comes our heart becomes ours. We can be in harmony with each other if it is not right to
 It is not always easy to love others who don't have a history. But once you know the difference between what one sees in each other, that
 I don't know what to do with people. I can think about all of the things in life, and if you have something that makes them feel
 When you're in love, don't be afraid to let yourself down.
 Love is not a competition. It's what you get, and it doesn't mean that people are always in love or e

In [ ]:
torch.save(model.state_dict(), "bot.pt")